importing libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import re
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Loading dataset

In [ ]:
dataset=pd.read_csv('hate_speech_detection .csv')
tweets=dataset.iloc[:,2].values
test=[dataset.iloc[1,2]] #for checking the working of clean 
y=dataset.drop(['tweet','id'],axis='columns')
y.value_counts()

label
0        3000
1        2242
dtype: int64

Cleaning text

In [ ]:
def clean(text):
  STOPWORDS.update({'is','the','of','to','on','so'})
  clean_tweet=[]
  for i in range(len(text)):
    sample=re.sub(r'[^a-zA-Z\n]'," ",text[i])
    sample=re.sub(r"[.,'\"@#!?]"," ",sample)
    sample=re.sub('user'," ",sample)
    sample=sample.lower()
    sample=sample.split()
    lm=WordNetLemmatizer()
    for word in sample:
      word=lm.lemmatize(word)
    sample=list(set(sample)-STOPWORDS)
    sample=" ".join(sample)
    clean_tweet.append(sample) 
  return clean_tweet
type(clean(tweets))#Showing the 10 clean tweets from dataset 

list

In [ ]:
def get_predicton(tweet):
  lis=[[]]
  lis[0].append(tweet)
  tweet=lis
  tweet=clean(tweet[0])
  lol=cv.transform(clean(tweet))
  pred=rf.predict(lol)
  if pred[0]==1:
    result='Toxic tweet'
  else:
    result='Non-Toxic tweet'

  return result

Creating tokens

In [ ]:
cv=CountVectorizer(max_features=5000)
x=cv.fit_transform(clean(tweets)).toarray()

Tackling imbalence in dataset

In [ ]:
sm=SMOTE()
x_sm,y_sm=sm.fit_resample(x,y)

In [ ]:
y_sm=y_sm['label'].values #making it nd array to feed the model

Splitting dataset in to test and training set

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x_sm,y_sm,test_size=0.3,random_state=0)
type(ytrain)

numpy.ndarray

Training model on training set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,criterion='gini',max_features='log2') #best parameter from Grid search is used in this model
rf.fit(xtrain,ytrain)
ypred=rf.predict(xtest)

Grid Search with cross validation

In [ ]:
param=[{
    'n_estimators':[50,100,120],
    'criterion':['gini','entropy'],
    'max_features':['sqrt','log2','']}]
gw=GridSearchCV(estimator=rf,param_grid=param,scoring='accuracy',cv=5)
gw.fit(xtrain,ytrain)
best_acc=gw.best_score_
beat_param=gw.best_params_
print('Best Accuracy: {:.2f} %'.format(best_acc*100) )
print('best param:',beat_param )


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 459, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/para

Best Accuracy: 84.88 %
best param: {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100}


Checking accuracy

In [ ]:
from sklearn.metrics import classification_report
print('Classification Report of the Random Forest model: \n',classification_report(ytest,ypred))

Classification Report of the Random Forest model: 
               precision    recall  f1-score   support

           0       0.87      0.80      0.83       895
           1       0.81      0.88      0.84       905

    accuracy                           0.84      1800
   macro avg       0.84      0.84      0.84      1800
weighted avg       0.84      0.84      0.84      1800



Testing model on a random tweet

In [ ]:
print(get_predicton("@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run"))

Non-Toxic tweet
